In [21]:
import os

# 1. Khai báo tên model, pretrain_model và file cần làm việc

In [28]:
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py' #Tên file convert xml sang tfrecord
LABEL_MAP_NAME = 'label_map.pbtxt' #Tên file label
MAIN_FOLDER_PATH = os.getcwd()
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' # Tên của custom model
MAIN_FOLDER_PATH

'D:\\TestClone\\DetectWearingHelmetDjangoWebsite'

# 2. Khai báo các đường dẫn thư mục làm việc

In [23]:
paths = {
    'WORKSPACE_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace'), #Folder lảm việc, lưu dữ liệu khi train, valid và test
    'SCRIPTS_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow','scripts'), #Folder chạy các file script liên quan đến convert ảnh
    'APIMODEL_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow','models'), #Folder chứa tensorflow object detection api
    'ANNOTATION_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace','annotations'), #Folder chứa các file config và file record
    'IMAGE_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace','images'), #Folder chứa ảnh
    'MODEL_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace','models'), #Folder chứa dữ liệu model khi làm việc(train, valid, test)
    'PRETRAINED_MODEL_PATH': os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','pre-trained-models'), #Folder chứa pretrain model
    'CHECKPOINT_PATH': os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), #Folder chứa các checkpoint
    'EXPORTED_MODEL_PATH': os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','exported-models', CUSTOM_MODEL_NAME), #Folder chứa model được lưu lại
    'PROTOC_PATH':os.path.join(MAIN_FOLDER_PATH,'Tensorflow','protoc') #Folder chứa protocol
}
paths

{'WORKSPACE_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\workspace',
 'SCRIPTS_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\scripts',
 'APIMODEL_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\models',
 'ANNOTATION_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\workspace\\annotations',
 'IMAGE_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\workspace\\images',
 'MODEL_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\workspace\\models',
 'PRETRAINED_MODEL_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\workspace\\pre-trained-models',
 'CHECKPOINT_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\workspace\\models\\my_ssd_mobnet',
 'EXPORTED_MODEL_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\workspace\\exported-models\\my_ssd_mobnet',
 'PROTOC_PATH': 'D:\\TestClone\\DetectWearingHelmetDjangoWebs

# 3. Khai báo các đường dẫn file làm việc

In [24]:
files = {
    'PIPELINE_CONFIG':os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'), #Path file pipelineconfig
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), #Path đến file convert từ xml sang tfrecord 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME) #Path label_config
}
files

{'PIPELINE_CONFIG': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\workspace\\models\\my_ssd_mobnet\\pipeline.config',
 'TF_RECORD_SCRIPT': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\scripts\\generate_tfrecord.py',
 'LABELMAP': 'D:\\TestClone\\DetectWearingHelmetDjangoWebsite\\Tensorflow\\workspace\\annotations\\label_map.pbtxt'}

## Khởi tạo các đường dẫn thư mục và tải các thư viện cần thiết

In [5]:
for path in paths.values():
    if not os.path.exists(path):
          !mkdir {path}

In [11]:
!pip install wget

  Using cached wget-3.2-py3-none-any.whl


In [18]:
import wget

### Clone model tensorflow và object_detection

In [14]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {os.path.join(paths['APIMODEL_PATH'])}

### Cài đặt và giải nén protocol buffer, cài đặt và cấu hình môi trường protocol buffer cho việc sử dụng TensorFlow Object Detection API.

In [15]:
# Tải Protocol
if len(os.listdir(paths['PROTOC_PATH'])) == 0:
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    os.system(f"move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}")
    os.system(f"cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip")

100% [..........................................................................] 1468733 / 1468733

In [16]:
# Thêm đường dẫn folder bin vào biến môi trường
os.environ['PATH'] += os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))

In [ ]:
#Kiểm tra đường dẫn hợp lệ hay chưa, đảm bảo đường dẫn của ...\Tensorflow\protoc\bin ở dòng cuối
os.environ['PATH']

In [18]:
# Compile protobuf files and install all required libraries
os.system(f"cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install")

1

In [19]:
os.system(f"cd Tensorflow/models/research/slim && pip install -e .")

0

In [ ]:
#Tải các thư viện cần thiết
!pip install -r requirements.txt

In [34]:
# Chỉnh sửa lại tfexample_decoder.py để phù hợp với tensorflow 2.15.0
import shutil 
import re
original_path = os.path.join(MAIN_FOLDER_PATH, 'myvenv\\Lib\\site-packages\\tf_slim-1.1.0-py3.11.egg\\tf_slim\\data\\tfexample_decoder.py')
with open (original_path, 'r') as file: 
    content = file.read()
content = re.sub(r'import abc', 'import tensorflow as tf\n\nimport abc', content)
content = re.sub(r'control_flow_ops.case', 'tf.case', content) 
content = re.sub(r'control_flow_ops.cond', 'tf.compat.v1.cond', content)
with open(original_path, 'w') as file: 
    file.write(content)

### Kiểm tra tất cả thư viện đã tải về thành công

In [10]:
import tensorflow as tf

In [15]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2024-05-29 21:29:57.976990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

Running tests under Python 3.11.9: d:\TestClone\DetectWearingHelmetDjangoWebsite\myvenv\Scripts\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac

W0529 21:30:04.988887  6708 module_wrapper.py:149] From d:\TestClone\DetectWearingHelmetDjangoWebsite\myvenv\Lib\site-packages\keras\src\backend.py:873: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

2024-05-29 21:30:05.008819: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE SSE2 SSE3 SSE4.1 SSE4.2 AVX2 AVX_VNNI FMA, in other op

In [16]:
import object_detection

### Cài đặt và giải nén pretrain_model

In [29]:
if len(os.listdir(paths['PRETRAINED_MODEL_PATH'])) == 0:
    wget.download(f'{PRETRAINED_MODEL_URL}')
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

100% [........................................................................] 20518283 / 20518283        1 file(s) moved.


x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/saved_model.pb
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/variables/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/variables/variables.index
